In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import re
import joblib
import string

In [3]:
fake = pd.read_csv('Fake.csv')
true = pd.read_csv('True.csv')

In [4]:
fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [5]:
true.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [6]:
fake['class']=0
true['class']=1

In [7]:
data = pd.concat([fake,true],axis = 0)

In [38]:
data.sample(5)

,index,text,class
27853,4372,CAIRO (Reuters) - Jordan said on Friday a U.S....,1
40781,17300,MADRID (Reuters) - Spain s High Court said on ...,1
19055,19055,Obama can be seen addressing a crowd about rel...,0
11432,11432,,0
44580,21099,LONDON (Reuters) - Britain will pledge on Mond...,1


In [ ]:
data = data.drop(["title","subject","date"], axis = 1)

In [35]:
data.reset_index(inplace=True)

In [39]:
data.drop(['index'],axis = 1,inplace=True)

In [47]:
data.sample(5)

,text,class
4035,Most political campaigns prefer to spend the b...,0
17267,"George, Barack and Hillary the axis of evil Pr...",0
6822,Twitter users have been annihilating Donald Tr...,0
15345,We re thinking Trump shouldn t give the rating...,0
20990,Leave it to a Democrat to waste everyone s tim...,0


In [ ]:
def claen_text(text):
    text = text.lower()
    text = re.sub('\[.*?\]',"",text)
    text = re.sub("\\W"," ",text)
    text = re.sub("https?:://\S+","",text)
    text = re.sub("<.*?>+","",text)
    text = re.sub("[%s]" % re.escape(string.punctuation),"",text)
    text = re.sub("\w*\d\w*","",text)
    return text

In [51]:

data["text"] = data["text"].apply(claen_text)

In [53]:
x=data["text"]
y=data["class"]

xtrain, xtest, ytrain, ytest = train_test_split(x,y,test_size=0.25,random_state=42)

In [54]:
vectorizer = TfidfVectorizer()
xv_train = vectorizer.fit_transform(xtrain)
xv_test = vectorizer.transform(xtest)

In [55]:
lr = LogisticRegression()
lr.fit(xv_train,ytrain)

LogisticRegression()

In [56]:
prediction = lr.predict(xv_test)
lr.score(xv_test,ytest)

0.9858351893095768

In [57]:
print(classification_report(ytest,prediction))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5895
           1       0.98      0.99      0.99      5330

    accuracy                           0.99     11225
   macro avg       0.99      0.99      0.99     11225
weighted avg       0.99      0.99      0.99     11225



In [58]:
joblib.dump(vectorizer,"vectorizer.jb")
joblib.dump(lr,"lr_model.jb")

['lr_model.jb']